In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from parsel import Selector
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from facebook_scraper import get_posts ,get_posts_by_search
from multiprocessing import Pool, cpu_count
import re
from requests import get
import pandas as pd
import numpy as np 
import time
import os
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
from datetime import datetime

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/joelbess/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def typeMaison(publication,bag_app,bag_mais):
     dicoType={}
     count=1
     for word in publication:
        if word in bag_app:
            dicoType[count]='appartement'
            count=count+1  
        elif word in bag_mais:
            dicoType[count]='maison Individuel'
            count=count+1  
     return dicoType

In [4]:

def check(post_elmt):
    post_elmt.replace(".","")
    post_elmt = re.findall("\d+", post_elmt)
    if len(post_elmt) != 0 :
        post_elmt = post_elmt[0]  
        return post_elmt
    return 0

def montant_loyer_piece(sac_mot, post):
    dico_montant = {}
    post_suiv = ""
    cpt=0
    for i in range(0,len(post)):
        loyer=0
        if post[i] in sac_mot:
            # break
            post_pred = post[i-1]
            if i != (len(post)-1):
                post_suiv = post[i+1]
            loyer = check(post_pred)
            if loyer!=0:
                    cpt+=1
                    dico_montant[cpt]= loyer
            if i != (len(post)-1):        
                if ((loyer == 0) and (i != (len(post)-2))):
                    loyer = check(post[i+1])
                    if loyer!=0:
                        cpt+=1
                        dico_montant[cpt]= loyer
            
        if loyer==0:
            val=re.findall("[a-zA-Z]+", post[i])
            if len(val)!=0:
                if val[0].upper() in sac_mot:
                    loyer= check(post[i])
                    if loyer!=0:
                        cpt=cpt+1
                        dico_montant[cpt]= loyer

    return dico_montant


In [5]:
def standing(sac,eco,haut,moyen):
    dicoStanding = dict()
    k = 1
    for i in sac:
        if i in eco:
            dicoStanding[k] = "econonmique"
            k += 1
            continue
        if i in haut:
            dicoStanding[k] = "haut"
            k += 1
            continue
        if i in moyen:
            dicoStanding[k] = "moyen"
            k += 1     
    return dicoStanding

In [6]:
immeuble_sac_mot =['REZ','APPARTEMENT','BALCON','APPARTEMENTS','APARTEMENT','APARTEMENTS', 
           'APPATEMENT','APPATEMENTS', 'APPAT', 'APPATS','IMMEUBLE',
           'IMMEUBLES','IMEUBLE','IMMEBLE',  'IMMEBLES','IMEBLE', 'IMEBLES','1ER','2EM','ETAGE']
villa_sac_mot=["VILLA","BASSE","VILA",'VILLAS','VILAS']
piece_sac_mot=["PIECE","PIECES","PIEC",'PCS']
montant_loyer_sac_mot = ["PRIX","LOYER", "FCFA","F","CFA","FR","MIL","MILLE","MILLES",'XOF','FA','€','$']
motCleLocationValide=["LOYER","LOUER","LOUE","LOU","LOCATION","LOCASSION","LOCASION","VISITE","CAUTION","CAUSION","CAUSSION","COSION","COSSION"]
motCleLocationExclu=["VENTE","VENT","TERRAIN","TERRIN","TERAIN","FONCIER","ACHETER","ACHETE"]

In [7]:
from unidecode import unidecode
import re
import codecs
def nettoyage(compltexte):
    compltexte=str(compltexte)
    compltexte = compltexte.replace("*", "")
    compltexte = compltexte.replace(",", "")
    compltexte = compltexte.replace(".", "")
    compltexte = re.sub(r"[^\w\s]", " ", compltexte) # supprime les caractères spéciaux et symboles
    listMot = compltexte.split() 
    listMot = [mot.upper() for mot in listMot]
    listMot = [unidecode(mot) for mot in listMot]
    return listMot

    


In [8]:

def first(postText):
    listTraite = nettoyage(postText)
    val=[]
    for mot in listTraite:
            if mot in motCleLocationValide:
                val= listTraite                
            elif mot in motCleLocationExclu:
                val=  []
            else:
                val= listTraite
    return val

        

In [9]:
database = pd.read_csv("../data/riviera_mpouto.csv")

In [10]:
database1 = pd.read_csv("../data/riviera.csv") 

In [11]:
database2 = pd.read_csv("../data/mpouto.csv") 

In [12]:
def traitement(database,localite):
    format = '%Y-%m-%d %H:%M:%S'
    baseFinal =[]
    for ligne in range(database.shape[0]):
        nuplet = database.iloc[ligne,:]
        publicationOk=first(nuplet["post_text"])
        if len(publicationOk) !=0:    
            typeM= typeMaison(publicationOk,immeuble_sac_mot,villa_sac_mot)
            piece = montant_loyer_piece(piece_sac_mot,publicationOk)
            montant = montant_loyer_piece(montant_loyer_sac_mot,publicationOk)
            if len(typeM)==1 and len(piece)==1 and len(montant)==1:
                ligneAEcricre = {}
                ligneAEcricre["username"] = nuplet["username"]
                ligneAEcricre["post_url"] = nuplet["post_url"]
                temps = datetime.strptime(str(nuplet["time"]),format)
                ligneAEcricre["jour"] = temps.day
                ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                ligneAEcricre["mois"] = temps.month
                ligneAEcricre["annee"] = temps.year
                ligneAEcricre["localite"] = localite
                ligneAEcricre["type"] = list(typeM.values())[0]
                ligneAEcricre["montant"] = list(montant.values())[0]
                ligneAEcricre["nbPiece"] = list(piece.values())[0]
                baseFinal.append(ligneAEcricre)
            if len(typeM)==0 and len(piece)==1 and len(montant)==1:
                ligneAEcricre = {}
                ligneAEcricre["username"] = nuplet["username"]
                ligneAEcricre["post_url"] = nuplet["post_url"]
                temps = datetime.strptime(str(nuplet["time"]),format)
                ligneAEcricre["jour"] = temps.day
                ligneAEcricre["mois"] = temps.month
                ligneAEcricre["annee"] = temps.year
                ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                ligneAEcricre["localite"] = localite
                ligneAEcricre["type"] = ""
                ligneAEcricre["montant"] =list( montant.values())[0]
                ligneAEcricre["nbPiece"] = list(piece.values())[0]
                baseFinal.append(ligneAEcricre)
            if len(typeM)>1 and len(piece)>1 and len(montant)>1:
                for ii in range(min([len(typeM),len(piece),len(montant)])):
                    ligneAEcricre = {}
                    ligneAEcricre["username"] = nuplet["username"]
                    ligneAEcricre["post_url"] = nuplet["post_url"]
                    temps = datetime.strptime(str(nuplet["time"]),format)
                    ligneAEcricre["jour"] = temps.day
                    ligneAEcricre["mois"] = temps.month
                    ligneAEcricre["annee"] = temps.year
                    ligneAEcricre["imageMaisonLien"] = nuplet["images_lowquality"]
                    ligneAEcricre["localite"] = localite
                    ligneAEcricre["type"] = list(typeM.values())[ii]
                    ligneAEcricre["montant"] = list(montant.values())[ii]
                    ligneAEcricre["type"] = list(typeM.values())[ii]
                    baseFinal.append(ligneAEcricre) 
        
    return pd.DataFrame(baseFinal).to_csv("../data/baseFinale_"+localite+".csv") 

        
            


In [22]:
traitement(database,"riviera m'pouto") 
traitement(database1,"riviera")  

In [13]:
traitement(database1,"m'pouto")